<h1 style="font-family:verdana;"> <center>Human Protein Atlas - Single Cell Classification🎖 </center> </h1>

<h3><center style="color:#159364; font-family:cursive;">Inference Notebook 🦾</center></h3>


### Do check-out for [Traning NOTEBOOK](https://www.kaggle.com/akhileshdkapse/hpa-cell-classification-efficientnets-tpu-training) :-)

In [ ]:
########## Hyperparameters ##############
img_size= (300,300)
classes= 19
seed= 32
batch_size= 14
val_split= 0.12
lr= 0.001

In [ ]:
import pandas as pd 
import numpy as np
import os

import tensorflow as tf

In [ ]:
# Reading images
def build_decoder(with_labels=True, target_size=img_size, ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
            
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


# Augmenting images
def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        #img= tf.image.random_crop(img, size, seed=None, name=None)
        img= tf.image.random_brightness(img, 0.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

In [ ]:
# TPU or GPU detection
def auto_select_accelerator():
    """
    Reference: 
        * https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
        * https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
    """
    try:  # detect TPUs
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  ## detect TPUs
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
        #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
        #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines
        
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    return strategy



In [ ]:
test_decoder = build_decoder(with_labels=False, target_size= img_size)

In [ ]:
def Build_dataset(paths, labels= None, batch= batch_size,
                  decode_fn=test_decoder, augment_fn=None,
                  augment= False, repeat= True, shuffle= seed):
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(batch).prefetch(AUTO)
    
    return dset

In [ ]:
Base_dir = "hpa-single-cell-image-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * batch_size

In [ ]:
tpu_bsize= batch_size * strategy.num_replicas_in_sync
tpu_bsize

In [ ]:
sub_df = pd.read_csv('/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')
test_paths = '../input/hpaimage512-data/TarName/test/' + sub_df['ID'] + '.jpg'

In [ ]:
# Get the multi-labels
label_cols = sub_df.columns[1:]

#dtest = Build_dataset(paths= test_paths, labels= None, augment= False, repeat=False, shuffle=False)
dtest = Build_dataset(paths= test_paths, labels= None, augment= False, repeat=False, shuffle=False)

In [ ]:
img_size = img_size[0]
seed= 35
#from tensorflow.keras import layers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# ML tools 
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Activation, Conv2D, Lambda, \
                            Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, multiply
from keras.optimizers import Adam
from tensorflow.keras import Model
# import tensorflow.keras.applications.efficientnet as efn
from tensorflow.keras.applications import Xception
import os
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
with strategy.scope():
    model= tf.keras.models.load_model('../input/hpa-cell-classification-efficientnets-tpu-training/hpa_effb3.h5')

In [ ]:
model.summary()

In [ ]:
preds = model.predict(dtest, verbose=1)
preds.shape

In [ ]:
sub_df['INDEX']= list(range(0, sub_df.shape[0]))
sub_df.head()

In [ ]:
encoded_df= pd.read_csv('../input/generated-mask/encoded_csv.csv')
encoded_df.drop(['PredictionString'], 1, inplace=True)
encoded_df.head()

In [ ]:
df=pd.merge(sub_df, encoded_df, on='ID', how='left')
df.head()

In [ ]:
def thresh(i, thr=0.5):
    p= preds[i]
    string=''
#     for ind, confi in enumerate(p):
#         if confi >thr:
#             x= '{} {:.6} '.format(ind, confi) + df.encode[i][2:-1] +' '
#             string+= x
    string+= '0 1 ' + df.encode[i][2:-1] +' '
    return string

In [ ]:
sub_df.PredictionString= sub_df.INDEX.apply(thresh)
sub_df.drop(['INDEX'], 1, inplace=True)
sub_df.head(5)

In [ ]:
sub_df.PredictionString[0].split(' ')

In [ ]:
#sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

In [ ]:
sub_df.to_csv('submission.csv', index=False)
sub_df.head()



## 🌄 Thanks for Reading

![](https://i.gifer.com/7ImI.gif)



<div class="alert alert-block alert-info" style="font-size:20px; font-family:verdana;">
 <a target="_blank" style="color:orange;">Do UPVOTE for more Motivation🤞</a>
</div>



<hr><hr><hr>

<hr>